<a href="https://colab.research.google.com/github/pireylow/LSTM-keras-stock-price-prediction/blob/main/Multiple_Tickers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install yfinance


In [17]:
#import necessary modules
import random
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import yfinance as yf
yf.pdr_override()

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter



In [18]:
#set stock information
ticker = 'SPY'
startdate = '2018-01-01'
enddate = '2023-01-01'

start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)
sdata = pdr.get_data_yahoo(ticker, start=start, end=end)

columnsTitles=['Open','High','Low','Close','Volume','Adj Close']
sdata=sdata.reindex(columns=columnsTitles)

close=sdata['Close']


[*********************100%***********************]  1 of 1 completed


In [19]:
def MA_strategy1(close, fast_val, slow_val, sma = True):

  if sma:
    short_rolling = close.rolling(window=fast_val).mean()
    long_rolling = close.rolling(window=slow_val).mean()
  else:
    short_rolling = close.ewm(span=fast_val, adjust=False).mean()
    long_rolling = close.ewm(span=slow_val, adjust=False).mean()

  time_counter = 250
  profit = 0
  FF_loop_flag = True
  while FF_loop_flag:

    #FF to first up cross and save the entry price
    FF_loop_flag = True
    while FF_loop_flag and short_rolling[time_counter] <= long_rolling[time_counter]:
      time_counter += 1
      if time_counter >= len(short_rolling):
        FF_loop_flag = False
        time_counter -= 1
    entry_price = close[time_counter]

    #FF to first close below slow MA and save the entry price
    while FF_loop_flag and close[time_counter] >= long_rolling[time_counter]:
      time_counter += 1
      if time_counter >= len(short_rolling):
        FF_loop_flag = False
        time_counter -= 1
    exit_price = close[time_counter]
    profit = profit + (exit_price - entry_price)


    #FF to first MA down cross to set up for next buy signal
    while FF_loop_flag and short_rolling[time_counter] >= long_rolling[time_counter]:
      time_counter += 1
      if time_counter >= len(short_rolling):
        FF_loop_flag = False
        time_counter -= 1

  return profit


In [20]:


#set up slowperiod and fastperiod values
fastminval = 3
fastmaxval = 50
fastvec = list(range(fastminval, fastmaxval+1))
slowmult = [2.5,3,3.5,4,4.5,5]
slowlist = []
fastlist = []
for fastPeriod in fastvec:
  tmp_slow = []
  tmp_fast = []
  for multval in slowmult:
    tmp_slow.append(int(multval*fastPeriod))
    tmp_fast.append(fastPeriod)
  slowlist.append(tmp_slow)
  fastlist.append(tmp_fast)


#set stock information
tickers = ['SPY', 'DIA', 'QQQ', 'IWM', 'AAPL', 'TSLA', 'GS', 'MA', 'V']
startdate = '2018-01-01'
enddate = '2023-01-01'
start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)

#test some tickers
results = { }
for ticker in tickers:

  print(ticker)

  sdata = pdr.get_data_yahoo(ticker, start=start, end=end)
  columnsTitles=['Open','High','Low','Close','Volume','Adj Close']
  sdata=sdata.reindex(columns=columnsTitles)
  close=sdata['Close']

  #backtest each fastperiod/slowperiod pair
  net_profit_list = []
  for i in range(len(fastvec)):
    fastPeriod = fastvec[i]
    tmp_profit_list = []
    for slowPeriod in slowlist[i]:

      #print([fastPeriod,slowPeriod])
      tmp_profit_list.append(MA_strategy1(close, fastPeriod, slowPeriod, sma = False))

    net_profit_list.append(tmp_profit_list)

  #Surface plot of profitability ...
  fast_length=len(fastvec)
  slow_length=len(slowmult)
  X = np.zeros((fast_length,slow_length))
  Y = np.zeros((fast_length,slow_length))
  Z = np.zeros((fast_length,slow_length))
  for i in range(fast_length):
    for j in range(slow_length):
      X[i,j] = fastlist[i][j]
      Y[i,j] = slowlist[i][j]
      Z[i,j] = net_profit_list[i][j]
  maxptr = np.unravel_index(np.argmax(Z, axis=None), Z.shape)
  results[ticker] = [Z[maxptr], int(X[maxptr]), int(Y[maxptr])]





SPY
[*********************100%***********************]  1 of 1 completed
DIA
[*********************100%***********************]  1 of 1 completed
QQQ
[*********************100%***********************]  1 of 1 completed
IWM
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
GS
[*********************100%***********************]  1 of 1 completed
MA
[*********************100%***********************]  1 of 1 completed
V
[*********************100%***********************]  1 of 1 completed


In [21]:
results

{'SPY': [140.41998291015625, 28, 126],
 'DIA': [122.32997131347656, 5, 15],
 'QQQ': [164.62998962402344, 45, 157],
 'IWM': [49.029998779296875, 7, 21],
 'AAPL': [89.66755676269531, 4, 12],
 'TSLA': [322.94729137420654, 3, 7],
 'GS': [244.72999572753906, 3, 12],
 'MA': [51.03997802734375, 23, 92],
 'V': [27.449996948242188, 49, 147]}